## Thematic map (chorophlet) in Julia
Covid-19 cases per 100.000 inhabitants

#### Modules

In [1]:
# Loading modules
using CSV
using DataFrames
using Dates
using Shapefile
using ZipFile
using Plots
using ColorSchemes

#### Directories

In [2]:
# Create directorties
for dir in ["data", "downloads", "shapefiles", "output"]
    path = joinpath(pwd(), dir)
    if !ispath(path)
        mkpath(path)
    end
end

#### Covid data

In [3]:
# Download csv file
csv_url = "https://data.rivm.nl/covid-19/COVID-19_aantallen_gemeente_per_dag.csv"
csv_loc = joinpath(pwd(), "data", split(csv_url, "/")[end])
download(csv_url, csv_loc)

# Read data
covid = CSV.File(csv_loc, pool=false) |> DataFrame

# Select columns of interest
select!(covid, ["Date_of_publication", "Municipality_name", "Total_reported"])

# Drop NA's
dropmissing!(covid)

# Sum multiple data points for a municipality on the same date
covid = combine(groupby(covid, [:Date_of_publication, :Municipality_name]), :Total_reported .=> sum .=> :Total_reported);

#### Shapefiles

In [4]:
# Download shapefiles
zip_url = "https://download.cbs.nl/regionale-kaarten/wijkbuurtkaart_2023_v1.zip"
zip_loc = joinpath(pwd(), "downloads", split(zip_url, "/")[end])
if ~isfile(zip_loc)
    download(zip_url, zip_loc)
end

# Extract municipality (gemeente) files
prefix = "gemeente"
r = ZipFile.Reader(zip_loc)
for f in r.files
    file_name = split(f.name, "/")[end]
    if startswith(file_name, prefix)
        # println("Extracting: $(file_name)")
        open(joinpath(pwd(), "shapefiles", file_name), "w") do io
            write(io, read(f))
        end
        if split(file_name, ".")[end] == "shp"
            name_shapefile = file_name
            global path_shapefile = joinpath(pwd(), "shapefiles", name_shapefile)
            # println(path_shapefile)
        end
    end
end

# Read shapefile
table = Shapefile.Table(path_shapefile)

# Create geodataframe
gdf = table |> DataFrame

# Filter for land (i.e. not water)
filter!(:H2O => ==("NEE"), gdf)

# Select columns of interest
select!(gdf, ["GM_NAAM", "AANT_INW", "geometry"]);

#### Join dataframes

In [5]:
# Date of publication
Date_of_publication = Date(2022, 2, 22)

# Join data
df = leftjoin(
    gdf, 
    covid[covid.Date_of_publication .== Date_of_publication, ["Municipality_name", "Total_reported"]], 
    on = "GM_NAAM" => "Municipality_name"
)

# Add column
df.total_per_100k = df.Total_reported .* (100_000 ./ df.AANT_INW)

# Select columns of interest
select!(df, [:geometry, :total_per_100k]);

#### Plot thematic map (chorophlet)

In [6]:
plot_size = (600, 650);

##### Solution 1

In [7]:
# complete rows
df1 = filter(row -> !ismissing(row.total_per_100k), df)

# plot complete rows
p1 = plot(
    df1.geometry, 
    fill = palette(:inferno), 
    fill_z = reshape(df1.total_per_100k, 1, nrow(df1)), 
    linecolor = "white", 
    linewidth = .5, 
    title = "COVID-19 Cases per 100,000 Inhabitants\n$(string(Date_of_publication))", 
    axis = false, 
    ticks = false, 
    size = plot_size, 
)

# incomplete rows
df2 = filter(row -> ismissing(row.total_per_100k), df)

# plot incomplete rows
plot!(
    df2.geometry, 
    color = "grey", 
    linecolor = "white", 
    linewidth = .5, 
    axis = false, 
    ticks = false, 
    size = plot_size, 
)

savefig(p1 , joinpath("output", "fig1.png"))

"/Users/rene/Documents/Projects/Github/Corona/output/fig1.png"

![Figure 1](output/fig1.png)

##### Solution 2

In [8]:
# complete rows
df1 = filter(row -> !ismissing(row.total_per_100k), df)

# Covert column to integers
df1.total_per_100k = [Int(round(x, digits = 0)) for x in df1.total_per_100k]

# plot complete rows
p2 = plot(
    df1.geometry, 
    color = palette(:viridis), 
    fill_z = permutedims(df1.total_per_100k), 
    linecolor = "white", 
    linewidth = .5, 
    axis = false, 
    ticks = false, 
    title = "COVID-19 Cases per 100,000 Inhabitants\n$(string(Date_of_publication))", 
    size = plot_size, 
)

# incomplete rows
df2 = filter(row -> ismissing(row.total_per_100k), df)

# plot incomplete rows
plot!(
    df2.geometry, 
    color = "grey", 
    linecolor = "white", 
    linewidth = .5, 
    axis = false, 
    ticks = false, 
    size = plot_size, 
)

savefig(p2 , joinpath("output", "fig2.png"))

"/Users/rene/Documents/Projects/Github/Corona/output/fig2.png"

![Figure 1](output/fig2.png)

#### Cleanup

In [9]:
covid = nothing
table = nothing
gdf = nothing
df = nothing
df1 = nothing
df2 = nothing
p1 = nothing
p2 = nothing